# Практическая работа №3: Обработка выборочных данных. Нахождение интервальных оценок параметров распределения. Проверка статистической гипотезы о нормальном распределении

Выполнил студент гр. 1384 Шаганов Вячеслав.

## Цель работы

Получение практических навыков вычисления интервальных статистических оценок параметров распределения выборочных данных и проверки «справедливости» статистических гипотез.

## Основные теоретические положения

#### Интервальные оценки

1. Доверительный интервал для математического ожидания ($\mu$):

   При неизвестном среднеквадратичном отклонении $\sigma$, используется $t$-распределение:
   $   \left( \bar{x} - t_{\gamma/2} \cdot \frac{s}{\sqrt{n}}, \bar{x} + t_{\gamma/2} \cdot \frac{s}{\sqrt{n}} \right),$
   где $\bar{x}$ — выборочное среднее, $s$ — выборочное отклонение, $t_\gamma$ — квантиль распределения Стьюдента для уровня надежности $\gamma$.

2. Доверительный интервал для среднеквадратичного отклонения ($\sigma$):

   Используется $\chi^2$-распределение:
   $   \left( \sqrt{ \frac{(n-1)s^2}{\chi^2_{\gamma/2}}}, \sqrt{\frac{(n-1)s^2}{\chi^2_{1-\gamma/2}}} \right).$

#### Проверка гипотезы о нормальности

1. Критерий $\chi^2$ Пирсона:

   $   \chi^2_{\textit{observed}} = \sum_{i = 1}^k\frac{(n_i - n_i')^2}{n_i'},$
   где $n_i$ — наблюдаемая частота, $n_i'$ — теоретическая частота.

2. Сравнение:

   Определяется критическая точка $\chi^2_{\textit{crit}}$ при уровне значимости $\alpha$. Если $\chi^2_{\textit{observed}} > \chi^2_{\textit{crit}}$, данные нельзя считать нормально распределёнными.



## Постановка задачи

Для заданной надежности определить (на основании выборочных данных и результатов выполнения практической работы №2) границы доверительных интервалов для математического ожидания и среднеквадратичного отклонения случайной величины. Проверить гипотезу о нормальном распределении исследуемой случайной величины с помощью критерия Пирсона $\chi^2$. Дать содержательную интерпретацию полученным результатам.

## Выполнение работы

Перенесём результаты практическо работы №2 в данную работу: воссоздадим интервальный ряд 

In [1]:
import numpy as np

print_slice = lambda arr, k, prefix='': print(prefix, *arr[:k], '...', *arr[-k:], sep=' ')

with open('sample.txt', 'r') as f:
    sample = [[float(e) for e in s.split()] for s in f.readlines()]

sample = np.array(sample)
X, Y = sample[:, 0], sample[:, 1]

In [14]:
series = Y
ranked = np.sort(series)
var = np.array( [ranked[0]] + [ranked[i] for i in range(1, len(ranked)) if ranked[i] != ranked[i-1]] )
k = 1 + 3.31 * np.log10(len(series))
k = int(k) if int(k) % 2 == 1 else int(k) + 1
minval, maxval = var[[0, -1]]
h = (maxval - minval) / k

intervals = [[minval + i*h, minval + (i+1)*h] for i in range(k)]
for i in range(k):
    b = intervals[i]

bins = [np.average(b) for b in intervals] # Середины интервалов 
counts = np.array([
    len(   
        series[    # получаем элементы по фильтру
            (series < intervals[i][1])   # отбираем элементы, меньшие x_{i+1}, где [x_i, x_{i+1}) - интервал
            | ((i == k-1) & (series <= intervals[i][1]))   # учитываем, что последний интервал имеет нестрогую правую границу
        ]
    ) for i in range(k)
])

rel_counts = counts / len(series)

abs_freq = [counts[i] - (0 if i == 0 else counts[i-1]) for i in range(k)]
rel_freq = [rel_counts[i] - (0 if i == 0 else rel_counts[i-1]) for i in range(k)]

C = bins[len(bins) // 2]
u = (bins - C) / h
nu = [rel_freq * u**k for k in range(5)]
n_u1_4 = rel_freq * (u+1)**4
Ms = [ sum(nu[i]) for i in range(len(nu)) ]
m  = [
    1, 
    0,
    (Ms[2] - Ms[1]**2)*h**2,
    (Ms[3] - 3*Ms[2]*Ms[1] + 2*Ms[1]**3)*h**3,
    (Ms[4] - 4*Ms[3]*Ms[1] + 6*Ms[2]*Ms[1]**2 - 3*Ms[1]**4)*h**4
]
u_mean = Ms[1] * h + C
u_disp = m[2]
s2 = len(series) / (len(series)-1) * u_disp
s = np.sqrt(s2)
As = m[3] / s**3
E = m[4] / s**4 - 3

Приступим к работе. 

#### 1. Вычислить точность и доверительный интервал для математического ожидания при неизвестном среднеквадратичном отклонении при заданном объёме выборки для доверительной точности $ \gamma \in \{0.95, 0.99\} $. Сделать выводы.

Для построения доверительного интервала введём в рассмотрение с.в. $t = \frac{\bar{x} - a}{S/\sqrt{N}}$, где $a$ - математическое ожидание, $S$ - исправленное среднеквадратичное выборочное отклонение, $N$ - объем выборки, в нашем случае - кол-во элементов интервального ряда.

Данная с.в. распределена по закону Стьюдента с $N-1$ степенями свободы. Плотность данного распределения обозначим за $s_{N-1}(t)$, а фунцию распределения за $S_{N-1}(t)$. 

Так как $P(|\frac{\bar{x}-a}{S / \sqrt{N}}| < t_\gamma) = \int^{t_{\gamma}}_{-t_{\gamma}} s_{N-1}(t) dt = S_{N-1}(t_\gamma) - S_{N-1}(-t_{\gamma})$, то $t_\gamma$ можно определить как $t_\gamma = S_{N-1}^{-1}(\frac{1 + \gamma}{2})$.

Тогда доверительный интервал для математического ожидания будет иметь вид:

$\bar{x} \pm t_{\gamma, N-1} \cdot \frac{S}{\sqrt{N}}$, 
где $t_{\gamma, N-1}$ - квантиль закона Стьюдента с $N-1$ степенями свободы. 


In [24]:
from scipy import stats
gamma = 0.95
df = k-1
t_gamma = stats.t.ppf((1+gamma)/2, df)
assert abs( gamma - (stats.t.cdf(t_gamma, df) - stats.t.cdf(-t_gamma, df)) ) < 1e-12, r"Проверяем равна ли \gamma разности функций распределений"

trust_interval = [u_mean - t_gamma * s/np.sqrt(k), u_mean + t_gamma * s/np.sqrt(k)]
print(f"Доверительный интервал для мат. ожидания надежности {gamma}: [{trust_interval[0]}; {trust_interval[1]}]")

Доверительный интервал для мат. ожидания надежности 0.95: [108.94839825264516; 149.81255412830717]


In [25]:
gamma = 0.99
t_gamma = stats.t.ppf((1+gamma)/2, df)
assert abs( gamma - (stats.t.cdf(t_gamma, df) - stats.t.cdf(-t_gamma, df)) ) < 1e-12, r"Проверяем равна ли \gamma разности функций распределений"

trust_interval = [u_mean - t_gamma * s/np.sqrt(k), u_mean + t_gamma * s/np.sqrt(k)]
print(f"Доверительный интервал для мат. ожидания надежности {gamma}: [{trust_interval[0]}; {trust_interval[1]}]")

Доверительный интервал для мат. ожидания надежности 0.99: [98.42290072850616; 160.33805165244618]


Таким образом, мат. ожидание распределения данной выборки лежит в интервале [108.95; 149.81] с уровнем доверия 0.95 и в интервале [98.42; 160.34] с уровнем доверия 0.99. Наблюдаем, что с ростом надёжности интервал становится больше

#### 2. Для вычисления границ доверительного интервала для среднеквадратичного отклонения определить значение $ q $ при заданных $ \gamma $ и $ n $. Построить доверительные интервалы, сделать выводы.

Определим значение $ q $ при заданных $ \gamma $ и $ n $ по таблице.

Для $ \gamma = 0.95 $ и $ n = 7 $ (кол-во интервалов) значение $ q $ равно $ 0.92 $.

Тогда, так как $ q < 1 $, то доверительный интервал будет иметь вид:
$ \left( S (1-q); S (1+q) \right) $

In [ ]:
q = 0.92
sko_trust_interval = [s * (1-q), s * (1+q)]
print(f"Доверительный интервал для мат. ожидания надежности {gamma}: ({sko_trust_interval[0]}; {sko_trust_interval[1]})")

Доверительный интервал для мат. ожидания надежности 0.99: [1.7673933604594747; 42.41744065102741]


Для $ \gamma = 0.99 $ и $ n = 7 $ (кол-во интервалов) значение $ q $ равно $ 1.62 $.

Тогда, так как $ q > 1 $, то доверительный интервал будет иметь вид:
$ \left( 0; S (1+q) \right) $

In [31]:
q = 1.62
sko_trust_interval = [0, s * (1+q)]
print(f"Доверительный интервал для мат. ожидания надежности {gamma}: ({sko_trust_interval[0]}; {sko_trust_interval[1]})")

Доверительный интервал для мат. ожидания надежности 0.99: (0; 57.88213255504783)


#### 3. Проверить гипотезу о нормальности заданного распределения с помощью критерия $ \chi^2 $ (Пирсона). Для этого необходимо найти теоретические частоты и вычислить наблюдаемое значение критерия. Для удобства вычисления необходимо заполнить табл. 1.

Для начала пересчитаем границы интервалов, рассчитаем вероятности попадания в каждый из получившихся интервалов и, домножив их на общее количество значений в выборке, получим теоретические частоты.

In [40]:
from scipy.stats import norm
z = [intervals[i][0] for i in range(len(intervals)) ] + [ intervals[-1][1] ]
z = [(xi - u_mean)/s for xi in z]
z[0] = -np.inf
z[-1] = np.inf

p = [norm.cdf(z[i]) - norm.cdf(z[i-1]) for i in range(1, len(z))]
n = [len(series) * p[i] for i in range(len(p))]

Теперь можно вычислить наблюдаемое значение критерия по формуле 
$\chi^2_{observed} = \sum_{i=1}^{k} \frac{(n_i - n'_i)^2}{n'_i}$

In [53]:
residuals_squares = [(ni - ni_prime)**2 for ni, ni_prime in zip(abs_freq, n)]
relative_residuals_squares = [rs / ni_prime for rs, ni_prime in zip(residuals_squares, n)]
chi_squared = sum(relative_residuals_squares)
print(f"Наблюдаемое значение критерия Пирсона равно {chi_squared:.3f}")
freq_squares = [ni**2 for ni in abs_freq]
relative_freq_squares = [ni**2 / ni_prime for ni, ni_prime in zip(abs_freq, n)]

objs = [abs_freq, p, n, residuals_squares, relative_residuals_squares, freq_squares, relative_freq_squares]
'''for i in range(k):
    els = [round(arr[i], 5) for arr in objs]
    print(f"| {i+1}", f"[{intervals[i][0]}, {intervals[i][1]})", *els,'', sep=' | ')
print( *[round(sum(arr), 6) for arr in objs], '', sep=' | ')''';

Наблюдаемое значение критерия Пирсона равно 6.890


Значения, позволяющие вычислить значение критерия, представим в таблице:

##### Таблица №1 
|  $ i $       |  $ [x_{i}, x_{i+1}) $  |  $ n_i $  |  $ p_i $  |  $ n'_i $  |  $ (n_i - n'_i)^2 $  |  $ \cfrac{(n_i - n'_i)^2}{n'_i} $  |  $ n_i^2 $  |  $ \cfrac{n_i^2}{n'_i} $  |
|-------------|-----------------------|---------|---------|----------|--------------------|----------------------------------|-----------|--------------------------|
| 1 | [62.6, 79.15714285714286) | 3 | 0.0115 | 1.38038 | 2.62316 | 1.90032 | 9 | 6.51994 | 
| 2 | [79.15714285714286, 95.71428571428572) | 6 | 0.05227 | 6.27194 | 0.07395 | 0.01179 | 36 | 5.73985 | 
| 3 | [95.71428571428572, 112.27142857142857) | 11 | 0.15557 | 18.66824 | 58.80195 | 3.14984 | 121 | 6.4816 | 
| 4 | [112.27142857142857, 128.82857142857142) | 39 | 0.2707 | 32.48361 | 42.46332 | 1.30722 | 1521 | 46.82361 | 
| 5 | [128.82857142857142, 145.38571428571427) | 35 | 0.27558 | 33.0691 | 3.72839 | 0.11275 | 1225 | 37.04365 | 
| 6 | [145.38571428571427, 161.94285714285715) | 17 | 0.16414 | 19.69648 | 7.27103 | 0.36915 | 289 | 14.67267 | 
| 7 | [161.94285714285715, 178.5] | 9 | 0.07025 | 8.43024 | 0.32462 | 0.03851 | 81 | 9.60826 | 
|  $ \Sigma $ | - |120 | 1.0 | 120.0 | - | 6.889576 | - | 126.889576 | 

#### 4. Доказать, что $ \chi^2_{\textit{observed}} = \sum_{i = 1}^k\frac{n^2_i}{n'_i} - n. $ Проконтролировать корректность вычисления $ \chi^2_{\textit{observed}} $. 

$ \chi^2_{\textit{observed}} = \sum_{i=1}^{k} \frac{(n_i - n'_i)^2}{n'_i} $

$ = \sum_{i=1}^{k} \frac{n_i^2 - 2 n'_i n_i + {n'_i}^2}{n'_i} $
$ = \sum_{i=1}^{k} (\frac{n_i^2}{n'_i} - 2 \frac{n'_i n_i}{n'_i} + \frac{{n'_i}^2}{n'_i}) $

$ = \sum_{i=1}^{k} (\frac{n_i^2}{n'_i} - 2 n_i + n'_i) $
$ = \sum_{i=1}^{k} \frac{n_i^2}{n'_i} - 2 \sum_{i=1}^{k} n_i + \sum_{i=1}^{k} n'_i $

$ = \sum_{i=1}^{k} \frac{n_i^2}{n'_i} - 2 n + \sum_{i=1}^{k} n p_i $
$ = \sum_{i=1}^{k} \frac{n_i^2}{n'_i} - 2 n + n \sum_{i=1}^{k} p_i $

$ = \sum_{i=1}^{k} \frac{n_i^2}{n'_i} - 2 n + n \cdot 1 $
$ = \sum_{i=1}^{k} \frac{n_i^2}{n'_i} - n $

Что и требовалось доказать.

Пользуясь таблицей выше можно также вычислить $ \chi^2_{\textit{observed}} $: $\sum_{i=1}^{k} \frac{n_i^2}{n'_i} = 126.89$, $n = 120$.

$ \chi^2_{\textit{observed}} = \sum_{i = 1}^k\frac{n^2_i}{n'_i} - n = 6.89 $, как и было получено ранее

#### 5. По заданному уровню значимости $ \alpha = 0.05 $ и числу степеней свободы $ df $ найти критическую точку $ \chi^2_{\textit{крит}} $ и сравнить с наблюдаемым значением. Сделать выводы.

По таблице можем получить критическую точку $ \chi^2_{\textit{крит}} $ для уровня значимости $ \alpha = 0.05 $ и числа степеней свободы $ df = k - 3 = 4 $:

$$
\chi^2_{\textit{crit}} = 9.5
$$

Видим, что $ \chi^2_{\textit{observed}} < \chi^2_{\textit{crit}} $, значит, гипотезу нормальности отвергнуть нельзя.

## Выводы

В ходе работы были рассчитаны доверительные интервалы для математического ожидания и среднеквадратичного отклонения, которые расширяются с увеличением уровня надежности $\gamma$, обеспечивая более точную оценку параметров распределения. Проверка гипотезы о нормальности с помощью критерия $\chi^2$
 показала, что гипотезу о нормальном распределении отвергнуть нельзя, так как наблюдаемое значение критерия меньше критического. 